**Initializing libiraries:**

In [ ]:
!pip install openai
!pip install datasets
!pip install transformers
!pip install torch
!pip install pandas
!pip install huggingface_hub
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ["WANDB_DISABLED"] = "true"

**Ask Chatbot**

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_chatbot():
    model_name = "microsoft/DialoGPT-medium"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return tokenizer, model

def load_data_from_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def chat():
    tokenizer, model = load_chatbot()
    chat_history_ids = None

    # Load dataset
    dataset = load_data_from_json('dataset.json')

    while True:
        user_input = input("You: ")
        if not user_input:
            continue

        # Search for related data in dataset (if any)
        relevant_data = []
        for item in dataset:
            if user_input.lower() in item.get('question', '').lower():
                relevant_data.append(item.get('answer', ''))

        if relevant_data:
            print(f"Found related answers: {relevant_data}")

        new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt").to(model.device)
        chat_history_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

        response_ids = model.generate(
            chat_history_ids,
            max_length=chat_history_ids.shape[-1] + 50,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.6,
            top_k=40,
            top_p=0.85,
            repetition_penalty=1.2,
        )

        print("Chatbot:", tokenizer.decode(response_ids[:, chat_history_ids.shape[-1]:][0], skip_special_tokens=True), "\n")

chat()


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


Fine-tuning complete! The model has been saved.


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos tok

Chatbot: Hey! How are you?

